# カスミ 小霞

## sp計算方式 - 未進化前10~24等

sp = 285.72058843574945 + 等級 * 1.10000000 - 蔬果 * 1.00519442 + 食材 * 3.28584961 - 時間 * 3.85951158 + 持有上限 * 1.79438175 + 主技能內容 * 1.35008948 + 主技能等級 * 0 - 副技能內容 * 8.22966794 + 主技能影響程度 * 3.28584961 - 副技能影響程度 * 6.53446117 - 能力增加:主技能 * 3.66237375 - 能力增加:速度 * 3.28584961 - 能力增加:活力 * 6.76609455 + 能力增加:無 * 0 + 能力增加:發現 * 3.67242570 + 能力減少:主技能 * 0 + 能力減少:速度 * 3.66520055 - 能力減少:活力 * 3.28584961 + 能力減少:無 * 0 + 能力減少:發現 * 0

In [2]:
import sqlite3
import pandas as pd
import datetime

class SamePokemon():
    def __init__(self):
        self.conn = sqlite3.connect("pokemon_database.db")
        self.cursor = self.conn.cursor()
        self.cursor.execute('SELECT * FROM pokemon')
        self.data = self.cursor.fetchall()

    def get_data_as_dataframe(self):
        columns = [
            "id",
            "img_num",
            "name",
            "sp",
            "expertise",
            "level",
            "help_fruit",
            "help_fruit_num",
            "help_ingredient_1",
            "help_ingredient_num_1",
            "help_ingredient_2",
            "help_ingredient_num_2",
            "help_ingredient_3",
            "help_ingredient_num_3",
            "help_time",
            "help_max",
            "skill_main",
            "skill_main_num",
            "skill_main_level",
            "skill_second_1",
            "skill_second_num_1",
            "skill_second_2",
            "skill_second_num_2",
            "skill_second_3",
            "skill_second_num_3",
            "skill_second_4",
            "skill_second_num_4",
            "skill_second_5",
            "skill_second_num_5",
            "power_up",
            "power_down",
            "備註"
        ]

        df = pd.DataFrame(self.data, columns=columns)
        return df

# 創建一個物件
pokemon_instance = SamePokemon()

# 將資料以 DataFrame 格式讀取
pokemon_data = pokemon_instance.get_data_as_dataframe()

# 取得時間字串
time_strings = pokemon_data['help_time']

def convert_to_time_format(time_str):
    # 將時間字串轉換為時間物件
    time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    # 將時間物件轉換為整數，以時 * 10000 + 分 * 100 + 秒格式表示
    return time_obj.hour * 10000 + time_obj.minute * 100 + time_obj.second

# 將時間字串轉換為整數格式
time_converted = [convert_to_time_format(time_str) for time_str in time_strings]

# 更新 'help_time' 欄位
time_indices =list(range(13, 14))+list(range(15, 16))+ list(range(17, 19))+ list(range(28, 30))+ list(range(33, 34))+ list(range(43, 44))+ list(range(84, 90))

for i, idx in enumerate(time_indices):
    pokemon_data.at[idx, 'help_time'] = time_converted[i]


In [21]:
# 取y值
y = pokemon_data.iloc[list(range(13, 14))+list(range(15, 16))+ list(range(17, 19))+ list(range(28, 30))+ list(range(33, 34))+ list(range(43, 44))+ list(range(84, 90)), 3]

In [3]:
#取X值
X = pokemon_data.iloc[list(range(13, 14))+list(range(15, 16))+ list(range(17, 19))+ list(range(28, 30))+ list(range(33, 34))+ list(range(43, 44))+ list(range(84, 90)), [5, 7, 9] + list(range(14, 21))+ list(range(29,31))]

In [23]:
# 對 skill_main 進行 Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['skill_main_encoded'] = label_encoder.fit_transform(X['skill_main'])
# 除去原始的 skill_main 欄位
X = X.drop('skill_main', axis=1)


label_encoder = LabelEncoder()
X['skill_second_1_encoded'] = label_encoder.fit_transform(X['skill_second_1'])
# 除去原始的 skill_second_1 欄位
X = X.drop('skill_second_1', axis=1)


# 繼續對其他類別型欄位做 One-Hot Encoding
categorical_columns = ['power_up', 'power_down']

for col in categorical_columns:
    X = pd.concat([X, pd.get_dummies(X[col], prefix=col, drop_first=True)], axis=1)
    X = X.drop(col, axis=1)

In [89]:


label_encoder = LabelEncoder()
X['skill_second_2_encoded'] = label_encoder.fit_transform(X['skill_second_2'])
# 除去原始的 skill_second_2 欄位
X = X.drop('skill_second_2', axis=1)

label_encoder = LabelEncoder()
X['skill_second_3_encoded'] = label_encoder.fit_transform(X['skill_second_3'])
# 除去原始的 skill_second_3 欄位
X = X.drop('skill_second_3', axis=1)

label_encoder = LabelEncoder()
X['skill_second_4_encoded'] = label_encoder.fit_transform(X['skill_second_4'])
# 除去原始的 skill_second_4 欄位
X = X.drop('skill_second_4', axis=1)

label_encoder = LabelEncoder()
X['skill_second_5_encoded'] = label_encoder.fit_transform(X['skill_second_5'])
# 除去原始的 skill_second_5 欄位
X = X.drop('skill_second_5', axis=1)

KeyError: 'skill_second_1'

## 把數據分割成訓練集&測試集

In [55]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5,random_state=2)
# test_size=0.3 = 隨意抓取30%測試集
# random_state 重跑訓練值幾遍

In [56]:
from sklearn.linear_model import LinearRegression 
regressor=LinearRegression() # 初始化
model=regressor.fit(X_train,y_train)

In [57]:
y_predict=regressor.predict(X_test)

from sklearn.metrics import r2_score # 2的平方越大(越接近1)，多元線性回歸就越好
score1=r2_score(y_test,y_predict) # 比較訓練集&測試集之間的預測好不好

In [58]:
score1

0.4577909896234059

In [59]:
model.coef_ # y=b+7.03471165e*data1+1.20332163e*data2+3.96425367e*data3+1.53936312e*data4 -4.80880533e*data5

array([ 1.10000000e+01, -1.00519442e-02,  3.28584961e-03, -3.85951158e-15,
        1.79438175e-01,  1.35008948e-02,  0.00000000e+00, -8.22966794e-02,
        3.28584961e-03, -6.53446117e-04, -3.66237375e+00, -3.28584961e-03,
       -6.76609455e-03,  0.00000000e+00,  3.67242570e+00,  0.00000000e+00,
        3.66520055e+00, -3.28584961e-03,  0.00000000e+00,  0.00000000e+00])

In [60]:
model.intercept_ #b=74.48195300240218

285.72058843574945

In [61]:
model.score(X,y) # 再次評估loss值 =>跟score1比較=>數據可用

0.5829041507962376